<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_Variational-AutoEncoder/blob/main/TF2_0_VAE_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Function call
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import tensorflow as tf

from keras.layers import Input, Dense, Lambda, Reshape
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist


In [ ]:
# Hyper Parameters
batch_size = 16
original_dim = 784 #MNIST
latent_dim = 2
intermediate_dim = 256
epochs = 10
epsilon_std = 1.0 #For Sampling

In [ ]:
def sampling(args: tuple):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

  #Mean과 Log_Std로 Sampling한 최종 결과물 Return

In [ ]:
#Designing Encoder
x = Input(shape=(original_dim,), name="input")
h1 = Dense(intermediate_dim, activation='relu', name="encoding_1")(x)
h2 = Dense(int(intermediate_dim)/2, activation='relu', name="encoding_2")(h1)
h = Dense(int(intermediate_dim)/2, activation='relu', name="encoding_3")(h2)
z_mean = Dense(latent_dim, name="mean")(h)
z_log_var = Dense(latent_dim, name="log-variance")(h)
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
encoder = Model(x, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

In [ ]:
#Designing Decoder
input_decoder = Input(shape=(latent_dim,), name="decoder_input")
decoder_h1 = Dense(int(intermediate_dim/2), activation='relu', name="decoder_h1")(input_decoder)
decoder_h2 = Dense(intermediate_dim, activation='relu', name="decoder_h2")(decoder_h1)
decoder_h = Dense(int(intermediate_dim*1.5), activation='relu', name="decoder_h3")(decoder_h2)
x_decoded = Dense(original_dim, activation='sigmoid', name="flat_decoded")(decoder_h)
decoder = Model(input_decoder, x_decoded, name="decoder") #From size of Latent space to Original image
decoder.summary()

In [ ]:
output_combined = decoder(encoder(x)[2]) #Encoder의 3번째 Return값[2]이 Z[z_mean, z_log_var, z]
vae = Model(x, output_combined)
vae.summary()

In [ ]:
#Loss Function : KL-Divergence
kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae.add_loss(K.mean(kl_loss) / 784.)
vae.compile(optimizer='Adam', loss="binary_crossentropy")
vae.summary()

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.#Norm
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [ ]:
#Training
vae.fit(x_train, x_train,shuffle=False,epochs=epochs,batch_size=batch_size)

In [ ]:
#2D-Plotting을 위해선 Latent space를 2-Dimension으로밖에 잡을 수 없다는 한계점 존재
n = 1
m = 5
digit_size = 28 #MNIST default size
figure = np.zeros((digit_size * n, digit_size * m))
grid_x = norm.ppf(np.linspace(0.05, 0.95, n)) #linspace로 0~1사이 10등분하고 이걸 확률로본 다음, norm.ppf를 통해 Gaussian에서 각각 10개 값에 대한 pdf를 읽어옴
grid_y = norm.ppf(np.linspace(0.05, 0.95, m)) #여기서 읽어온 pdf값을 Mean, std로 가정하고 Decode진행

for i, yi in enumerate(grid_x): #for a,b in enumerate는 정수 a와 grid_x의 b값을 매칭해주는것. ex) (0,-1.64444)
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        print("xi : ",xi, "yi : ",yi)
        print("i : ",i, "j : ",j)
        x_decoded = decoder.predict(z_sample)
        print(i," Decode : ",x_decoded.shape)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='Greys_r')
plt.show()